##INSTALLING ALL THE **DEPENDENCIES**

In [1]:
pip install pytube


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 767.7 kB/s eta 0:00:00


In [2]:
pip install deepgram-sdk==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.2 MB/s eta 0:00:00


In [6]:
pip install python-dotenv

In [18]:
pip install elevenlabs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.4/110.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.2 MB/s eta 0:00:00


## **1.Download a youtube video using python script**

For downloading YouTube video , I used pytube library and saved it in the contents folder

In [4]:
from pytube import YouTube

# YouTube video URL
video_url = 'https://www.youtube.com/watch?v=cVsyJvxX48A'

# Download the video
yt = YouTube(video_url)
video = yt.streams.get_highest_resolution().download()

# Check the downloaded video path
print("Video downloaded successfully:", video)


Video downloaded successfully: /content/Learn English By Movies - The Pursuit Of Happyness (Scene With Subtitles).mp4


## **2. Extract the audio from the video**




For extracting audio from the video , I used moviepy.editor library

In [7]:
from moviepy.editor import VideoFileClip

def extract_audio(video_path, audio_path):
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)

# Use the function
video_path = "/content/Learn English By Movies - The Pursuit Of Happyness (Scene With Subtitles).mp4"
audio_path = "audio.mp3"
extract_audio(video_path, audio_path)


chunk:   1%|          | 30/3928 [00:53<1:56:14,  1.79s/it, now=None]

MoviePy - Writing audio in audio.mp3



chunk:   1%|          | 30/3928 [00:58<2:07:26,  1.96s/it, now=None]

MoviePy - Done.


In [16]:
from google.colab import userdata
deepgram=userdata.get('deepgram')
elevenlabs=userdata.get('elevenlabs')

##**3. Use Deepgram API to convert it into text**

the following code converts the audio file into text and returns a json file

In [3]:
from deepgram import Deepgram
import asyncio, json, os

dg = Deepgram(deepgram)

MIMETYPE = 'mp3'

DIRECTORY = '.'

params = {
    "punctuate": True,
    "model": 'general',
    "tier": 'nova'
}

# This function is what calls on the model to transcribe
def main():
    audio_folder = os.listdir(DIRECTORY)
    for audio_file in audio_folder:
        if audio_file.endswith(MIMETYPE):
          with open(f"{DIRECTORY}/{audio_file}", "rb") as f:
              source = {"buffer": f, "mimetype":'audio/'+MIMETYPE}
              res = dg.transcription.sync_prerecorded(source, params)
              with open(f"./{audio_file[:-4]}.json", "w") as transcript:
                  json.dump(res, transcript)
    return

main()

the below code reads the json file and stores the text in a **text** variable

In [11]:
OUTPUT = '/content/audio.json'



def get_transcript(transcription_file):
  with open(transcription_file, "r") as file:
        data = json.load(file)
        result = data['results']['channels'][0]['alternatives'][0]['transcript']
        result = result.split('.')
        for sentence in result:
          return ' '.join(result)

text=get_transcript(OUTPUT)

In [12]:
text

"Are we bringing it to the game? Yeah  I don't I don't wanna leave it  And maybe we're going to the game  Where are we going now? Say someone about my job  I don't understand  You don't understand what? I'll be you over to the game? I said possibly we're going to the game  Do you know what possibly means? Like probably? No  Probably means there's a good chance that we're gone  Possibly means, we might, we might not  What does probably mean? It means we have a good chance  And what does possibly mean? I know what it means  What does it mean? It means that we're not going to the game  Okay  How did you get so smart? It's too smart  Are we there? Yeah  Mister Rivan? Yes  How are you, sir? Chris Gardner  Deanwitter  Oh, hi  Hi  This is my son Christopher? Hi  Hey, Christopher  What what what are you doing up here? I came to apologize for missing our appointment the other day  Oh, you need to come up? Oh, no  Actually, we were in the neighborhood visiting a very close friend of ours and and

##**4. Use the text to generate audio using a voice of Eleven Labs**

In [28]:
from elevenlabs import play,save
from elevenlabs.client import ElevenLabs

client = ElevenLabs(
  api_key=elevenlabs, # Defaults to ELEVEN_API_KEY
)

audio = client.generate(
  text=text,
  voice="Bella"
)
play(audio,notebook=True)
